<a href="https://colab.research.google.com/github/NajimovOtabek/net_intrusion_detection/blob/master/How_to_Normalize.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Default

Accuracy: 0.9936641057818861

Brier Score: 0.005476696146055273

F1 Score: 0.9950603058587703


## Standardize

  from sklearn.preprocessing import StandardScaler

    def standardize(data, X_train):
      """
      Standardize the input data by subtracting the mean and dividing by the standard deviation.

      Args:
        data (numpy array): Input data to be standardized
        X_train (numpy array): Training data used to calculate the mean and standard deviation

      Returns:
        numpy array: Standardized data
      """
      scaler = StandardScaler()
      scaler.fit(X_train)
      return scaler.transform(data)


      X_train = standardize(X_train, X_train)
      X_test = standardize(X_test, X_train)

Accuracy: 0.32315028181392846

Brier Score: 0.25668962185635885

F1 Score: 0.42825245856226046


    from sklearn.preprocessing import StandardScaler

    def standardize(data):
      scaler = StandardScaler()
      scaler.fit(data)
      return scaler.transform(data)

    X = df_10.iloc[:, :-1].values
    y = df_10.iloc[:, -1].values

    X = standardize(X)


Accuracy: 0.9937012728632207
Brier Score: 0.005488090397172694
F1 Score: 0.9950890399166791

In [ ]:
# NF_ToN_IoT_v2

!gdown --id 1JNsyqlCwT8IVudqj3yZ1y6wgbG7me5Pq
!unzip /content/NF-ToN-IoT-v2.zip

/usr/local/lib/python3.10/dist-packages/gdown/cli.py:121: FutureWarning: Option `--id` was deprecated in version 4.3.1 and will be removed in 5.0. You don't need to pass it anymore to use a file ID.
  warnings.warn(
Downloading...
From: https://drive.google.com/uc?id=1JNsyqlCwT8IVudqj3yZ1y6wgbG7me5Pq
To: /content/NF-ToN-IoT-v2.zip
100% 185M/185M [00:07<00:00, 26.1MB/s]
Archive:  /content/NF-ToN-IoT-v2.zip
  inflating: NF-ToN-IoT-v2/NetFlow_v2_Features.csv  
  inflating: NF-ToN-IoT-v2/NF-ToN-IoT-v2.csv  


In [ ]:
import os
os.environ['CUDA_LAUNCH_BLOCKING'] = "1"
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import DataLoader, TensorDataset

from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import classification_report,accuracy_score, balanced_accuracy_score, f1_score, precision_score, recall_score, brier_score_loss
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import StratifiedShuffleSplit
from sklearn.preprocessing import LabelEncoder

from collections import Counter
import numpy as np
import os
import pandas as pd
import numpy as np
import pandas as pd
from os.path import join
import glob
import dask.dataframe as dd
from dask.diagnostics import ProgressBar

In [ ]:
df= pd.read_csv("/content/NF-ToN-IoT-v2/NF-ToN-IoT-v2.csv") # takes 46 seconds to load

df.head()

,IPV4_SRC_ADDR,L4_SRC_PORT,IPV4_DST_ADDR,L4_DST_PORT,PROTOCOL,L7_PROTO,IN_BYTES,IN_PKTS,OUT_BYTES,OUT_PKTS,...,TCP_WIN_MAX_IN,TCP_WIN_MAX_OUT,ICMP_TYPE,ICMP_IPV4_TYPE,DNS_QUERY_ID,DNS_QUERY_TYPE,DNS_TTL_ANSWER,FTP_COMMAND_RET_CODE,Label,Attack
0,192.168.1.193,49235,192.168.1.33,4444,6,0.0,155392,202,34552,149,...,45555,4805,0,0,0,0,0,0,1,ransomware
1,192.168.1.193,49228,192.168.1.152,1880,6,0.0,1600,40,35741,65,...,16425,237,0,0,0,0,0,0,0,Benign
2,192.168.1.152,0,192.168.1.193,0,1,0.0,212,2,0,0,...,0,0,771,3,0,0,0,0,0,Benign
3,192.168.1.169,65317,239.255.255.250,1900,17,0.0,165,1,0,0,...,0,0,0,0,0,0,0,0,0,Benign
4,192.168.1.79,60766,192.168.1.255,15600,17,0.0,63,1,0,0,...,0,0,0,0,0,0,0,0,0,Benign


In [ ]:
df = df.drop(columns=["Attack","IPV4_SRC_ADDR", "IPV4_DST_ADDR","SRC_TO_DST_SECOND_BYTES","DST_TO_SRC_SECOND_BYTES"])

df.head()

,L4_SRC_PORT,L4_DST_PORT,PROTOCOL,L7_PROTO,IN_BYTES,IN_PKTS,OUT_BYTES,OUT_PKTS,TCP_FLAGS,CLIENT_TCP_FLAGS,...,NUM_PKTS_1024_TO_1514_BYTES,TCP_WIN_MAX_IN,TCP_WIN_MAX_OUT,ICMP_TYPE,ICMP_IPV4_TYPE,DNS_QUERY_ID,DNS_QUERY_TYPE,DNS_TTL_ANSWER,FTP_COMMAND_RET_CODE,Label
0,49235,4444,6,0.0,155392,202,34552,149,24,24,...,95,45555,4805,0,0,0,0,0,0,1
1,49228,1880,6,0.0,1600,40,35741,65,24,16,...,6,16425,237,0,0,0,0,0,0,0
2,0,0,1,0.0,212,2,0,0,0,0,...,0,0,0,771,3,0,0,0,0,0
3,65317,1900,17,0.0,165,1,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,60766,15600,17,0.0,63,1,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [ ]:
df.shape

(16940496, 40)

In [ ]:


# Assuming df is your DataFrame
df = df.astype(float)

#take 10%
df_10 = df.sample(frac=0.05, random_state=42)

In [ ]:
df_10.shape

(847025, 40)

In [ ]:
# Assuming your DataFrame is named 'df'
df_10.to_csv('NF-ToN-IoT-v2-5_100.csv', index=False)


In [ ]:
from sklearn.preprocessing import StandardScaler

def standardize(data):
  scaler = StandardScaler()
  scaler.fit(data)
  return scaler.transform(data)



In [ ]:
X = df_10.iloc[:, :-1].values
y = df_10.iloc[:, -1].values

X = standardize(X)

# # Split the test data into validation and test sets
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=42, stratify=y)

In [ ]:
!pip install catboost


     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 98.7/98.7 MB 8.6 MB/s eta 0:00:00


In [ ]:
from catboost import CatBoostClassifier

# Initialize CatBoostClassifier
model = CatBoostClassifier(
    task_type='GPU', # This parameter enables GPU usage
    devices='0:1',   # Specify the GPU devices to use (0:1 for the first GPU)
    random_seed=42
)

# Train the model
model.fit(
    X_train,
    y_train,
    eval_set=(X_test, y_test),
    verbose=False # Set to True if you want to see the training progress
)

# Evaluate the model
print("Model is trained. You can now use it to make predictions or evaluate its performance.")


Model is trained. You can now use it to make predictions or evaluate its performance.


In [ ]:
from sklearn.metrics import accuracy_score, brier_score_loss, f1_score

# Make predictions
y_pred = model.predict(X_test)
y_pred_proba = model.predict_proba(X_test)[:, 1]  # Probabilities for the positive class

# Calculate Accuracy
accuracy = accuracy_score(y_test, y_pred)
print(f"Accuracy: {accuracy}")

# Calculate Brier Score
brier_score = brier_score_loss(y_test, y_pred_proba)
print(f"Brier Score: {brier_score}")

# Calculate F1 Score
f1 = f1_score(y_test, y_pred)
print(f"F1 Score: {f1}")


Accuracy: 0.9937012728632207
Brier Score: 0.005488090397172694
F1 Score: 0.9950890399166791


In [ ]:
# Convert data to PyTorch tensors
X_train_tensor = torch.tensor(X_train, dtype=torch.float32)
y_train_tensor = torch.tensor(y_train, dtype=torch.long)
X_test_tensor = torch.tensor(X_test, dtype=torch.float32)
y_test_tensor = torch.tensor(y_test, dtype=torch.long)

In [ ]:
# Create datasets and dataloaders
train_dataset = TensorDataset(torch.tensor(X_train, dtype=torch.float32), torch.tensor(y_train, dtype=torch.long))
test_dataset = TensorDataset(torch.tensor(X_test, dtype=torch.float32), torch.tensor(y_test, dtype=torch.long))

In [ ]:
class TransformerModel(nn.Module):
    def __init__(self, input_dim, num_classes, d_model, nhead, num_layers, dim_feedforward, dropout):
        super(TransformerModel, self).__init__()
        self.embedding = nn.Linear(input_dim, d_model)
        self.transformer = nn.Transformer(
            d_model=d_model,
            nhead=nhead,
            num_encoder_layers=num_layers,
            num_decoder_layers=num_layers,
            dim_feedforward=dim_feedforward,
            dropout=dropout
        )
        self.fc = nn.Linear(d_model, num_classes)

    def forward(self, x):
        x = self.embedding(x)
        x = x.unsqueeze(1)  # Add a fake batch dimension for the transformer
        x = self.transformer(x, x)  # Encoder-Decoder Self-Attention
        x = x.squeeze(1)
        x = self.fc(x)
        return x

In [ ]:
import torch
import numpy as np
from sklearn.metrics import balanced_accuracy_score, classification_report, brier_score_loss, average_precision_score, roc_auc_score

def evaluate_model(model, data_loader):
    model.eval()
    true_labels = []
    predicted_labels = []
    predicted_probs = []

    with torch.no_grad():
        for data, labels in data_loader:
            data = data.cuda()
            labels = labels.cuda()
            outputs = model(data)
            _, predicted = torch.max(outputs.data, 1)
            true_labels.extend(labels.cpu().numpy())
            predicted_labels.extend(predicted.cpu().numpy())
            predicted_probs.extend(torch.softmax(outputs, dim=1).cpu().numpy())

    predicted_probs = np.array(predicted_probs)  # Convert to numpy array

    accuracy = accuracy_score(true_labels, predicted_labels)
    balanced_accuracy = balanced_accuracy_score(true_labels, predicted_labels)

    macro_f1 = f1_score(true_labels, predicted_labels, average='macro')
    macro_precision = precision_score(true_labels, predicted_labels, average='macro')
    macro_recall = recall_score(true_labels, predicted_labels, average='macro')

    brier_score = brier_score_loss(true_labels, predicted_probs[:, 1])  # Assuming binary classification
    pr_auc = average_precision_score(true_labels, predicted_probs[:, 1])
    roc_auc = roc_auc_score(true_labels, predicted_probs[:, 1])

    print("Classification Report : \n", classification_report(true_labels, predicted_labels))

    return accuracy, balanced_accuracy, macro_f1, macro_precision, macro_recall, brier_score, pr_auc, roc_auc, true_labels, predicted_labels


In [ ]:
batch_size = 1024
train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True)
test_loader = DataLoader(test_dataset, batch_size=batch_size)


# Model Configuration
input_dim = X_train.shape[1]
unique_classes = np.unique(np.concatenate([y_train, y_test]))
num_classes = len(unique_classes)
model = TransformerModel(input_dim, num_classes, d_model=60, nhead=6, num_layers=4, dim_feedforward=128, dropout=0.1)
model = model.cuda()

# Loss and Optimizer
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=0.001)

/usr/local/lib/python3.10/dist-packages/torch/nn/modules/transformer.py:282: UserWarning: enable_nested_tensor is True, but self.use_nested_tensor is False because encoder_layer.self_attn.batch_first was not True(use batch_first for better inference performance)
  warnings.warn(f"enable_nested_tensor is True, but self.use_nested_tensor is False because {why_not_sparsity_fast_path}")


In [ ]:
# Training loop
for epoch in range(2):
    model.train()
    running_loss = 0.0
    for i, (data, labels) in enumerate(train_loader):
        data = data.cuda()
        labels = labels.cuda()

        # Forward pass
        outputs = model(data)
        loss = criterion(outputs, labels)

        # Backward and optimize
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

        running_loss += loss.item()

    print('Epoch [{}/5], Loss: {:.4f}'.format(epoch + 1, running_loss / len(train_loader)))

# Evaluation (replace with your evaluation function)
accuracy, balanced_accuracy, macro_f1, macro_precision, macro_recall, brier_score, pr_auc, roc_auc, true_labels, predicted_labels = evaluate_model(model, test_loader)
print('Accuracy: {:.4f}'.format(accuracy))
print('Brier Score: {:.4f}'.format(brier_score))
print('PR AUC: {:.4f}'.format(pr_auc))
print('ROC AUC: {:.4f}'.format(roc_auc))


Epoch [1/5], Loss: 0.1222
Epoch [2/5], Loss: 0.0661
Classification Report : 
               precision    recall  f1-score   support

           0       0.97      0.97      0.97    182768
           1       0.98      0.99      0.98    325447

    accuracy                           0.98    508215
   macro avg       0.98      0.98      0.98    508215
weighted avg       0.98      0.98      0.98    508215

Accuracy: 0.9800
Brier Score: 0.0153
PR AUC: 0.9986
ROC AUC: 0.9976
